# An Object-Oriented Method for Extracting Single-Object Aq-uaculture Ponds from 10 m Resolution Sentinel-2 Images on Google Earth Engine

This is the code used in the paper "An Object-Oriented Method for Extracting Single-Object Aquaculture Ponds from 10 m Resolution Sentinel-2 Images on Google Earth Engine", which can be run on Visual Studio Code or Google Colab after code edit.

Author: Boyi Li, Adu Gong*, Zikun Chen, Xiang Pan, Lingling Li, Jinglin Li, and Wenxuan Bao

*Correspondence: gad@bnu.edu.cn

Update: 2023/01/26

INPUT
      start: The start date to acquire Sentinel-2 surface reflectance images
      end: The end date to acquire Sentinel-2 surface reflectance images
      area_geometry: The spatial coverage of the study area
      proj: Spatial reference suitable for the study area
      export_path: The output path of the result file in your GEE assets
      
OUTPUT
      pond_seg: The single-object aquaculture ponds (SOAPs) extracted from the study area.

# Environment setting

In [1]:
import os
#todo 
# If you want to run this code on Visual Studio Code, please set the following parameters according to your PC information. 
# If you want to run the code on Google Colab, the following two lines are unnecessary.
os.environ['HTTP_PROXY'] = 'Please set this parameter according to your PC information'
os.environ['HTTPS_PROXY'] = 'Please set this parameter according to your PC information'

from utils import *

import geemap
import ee
from ee.batch import Export, Task
ee.Authenticate()
ee.Initialize()

In [ ]:
#todo Please set the following parameters according to your requirement
proj=ee.Projection('PROJCS["World_Cylindrical_Equal_Area",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Cylindrical_Equal_Area"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],UNIT["Meter",1.0]]')
start='2020-01-01'
end='2021-01-01'
area_geometry=ee.Geometry.Polygon(
  [[
    [79.72879371224818,7.593251546351282]
    ,[79.8702408582526,7.593251546351282]
    ,[79.8702408582526,8.001428733270046]
    ,[79.72879371224818,8.001428733270046]]],None,False)
export_path='Please set this parameter according to your GEE account'


# Water Pixel Identification

In [24]:
s2collection=collection_CP(area_geometry,start, end,True)
ndwicollection=ndwiCollection(s2collection)
ndwiimg_max=ndwicollection.max()
ndwiimg_z=z_score_img(ndwicollection,2)
ndwiimg_max_rn=ndwiimg_z.where(ndwiimg_z.gt(ndwiimg_max),ndwiimg_max)
ndwiimg_max_rn=ndwiimg_max_rn.clip(area_geometry)
water_mask=ndwiimg_max_rn.gt(0).selfMask()
water_mask=remove_small(water_mask,10)
ndwiimg_max_rn=ndwiimg_max_rn.mask(water_mask)

# Water Segmentation and Selection

In [ ]:
ndwi=ndwiimg_max_rn
scale_canny=5
n=3

i=0
ndwi0,canny_mask0=get_canny(ndwi,0,scale_canny)
water_object0=segment_by_tile(water_mask,canny_mask0,scale_canny,proj,area_geometry,i)
pond_object0,pond_not=sdd(water_object0,0,scale_canny)
object=pond_object0.merge(pond_not)
mask_t=water_mask.clip(object)
water_mask0=mask_adv(water_mask,mask_t)

In [ ]:
ndwii=ndwi0
canny_maski=canny_mask0
water_maski=water_mask0
pond_object=pond_object0
for i in range(1,n):
  ndwii,canny_mask_t=get_canny(ndwii,i,scale_canny)
  canny_maski=canny_maski.mask().add(canny_mask_t.mask()).gte(1).selfMask()
  water_objecti=segment_by_tile(water_maski,canny_maski,scale_canny,proj,area_geometry,i)
  pond_object_i,pond_not=sdd(water_objecti,i,scale_canny)
  pond_object=pond_object.merge(pond_object_i)
  object=pond_object_i.merge(pond_not)
  mask_t=water_maski.clip(object)
  water_maski=mask_adv(water_maski,mask_t)

# Aquaculture Ponds Extraction

In [28]:
obj=filter_area(pond_object,scale_canny,'b')
obj=obj.filterMetadata('d_area','equals',1)
obj=ndwiimg_max_rn.reduceRegions(collection=obj,reducer=ee.Reducer.median(),scale=scale_canny)
obj=obj.filterMetadata('median','not_less_than',0.15)
obj=get_crop(obj,scale_canny)
obj=obj.filterMetadata('crop','less_than',0.5)
obj=get_near_num(obj)
obj=obj.filterMetadata('near_num','not_less_than',3)
obj=get_buffer(obj)
pond_seg=obj

# Export

In [ ]:
export_table_toAsset(pond_seg,'pond_seg',export_path)